<a href="https://colab.research.google.com/github/SouravSangralSrv/SouravSangralSrv/blob/main/aftershoot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

# Define the paths
zip_path = '/content/aftershoot.zip' # The name of your existing ZIP file
# 🚨 FIX: Change the destination path to a NEW folder name
extract_path = '/content/aftershoot_extracted'

# Create the destination directory if it doesn't exist
# This is where the contents will go
os.makedirs(extract_path, exist_ok=True)

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted successfully to: {extract_path}")

Files extracted successfully to: /content/aftershoot_extracted


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

class WhiteBalancePredictor:
    """
    Delta-based predictor: Predict CHANGE from current values
    Key insight: We predict Temperature - currTemp and Tint - currTint
    """

    def __init__(self):
        self.image_model = None
        self.metadata_model = None
        self.ensemble_model = None
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.feature_cols = None
        self.feature_means = {}
        self.feature_medians = {}
        self.predict_deltas = True  # Predict changes instead of absolute values

    def load_and_preprocess_data(self, train_path='/content/aftershoot_extracted/dataset/Train'):
        """Load images and metadata with delta-based targets"""
        sliders_df = pd.read_csv(f'{train_path}/sliders.csv')

        # Exclude features
        exclude_features = [
            'id_global', 'Temperature', 'Tint',
            'touchTime', 'editTime', 'lastEditTime',
            'timestamp', 'date', 'time',
            'copyCreationTime', 'captureTime', 'grayscale'
        ]

        all_features = [col for col in sliders_df.columns
                       if col not in exclude_features]

        self.feature_cols = []
        for col in all_features:
            if sliders_df[col].dtype in ['float64', 'int64']:
                if sliders_df[col].std() > 1e-6:
                    self.feature_cols.append(col)
            elif sliders_df[col].dtype == 'object':
                unique_vals = sliders_df[col].nunique()
                if 1 < unique_vals < 100:
                    self.feature_cols.append(col)

        print(f"Using {len(self.feature_cols)} features: {self.feature_cols}")

        X_meta = sliders_df[self.feature_cols].copy()

        # KEY INSIGHT: Predict deltas (changes) instead of absolute values
        if 'currTemp' in self.feature_cols and 'currTint' in self.feature_cols:
            print("\n🎯 Using DELTA prediction (change from current values)")

            # Calculate deltas
            temp_delta = sliders_df['Temperature'] - sliders_df['currTemp']
            tint_delta = sliders_df['Tint'] - sliders_df['currTint']

            # Remove extreme outliers (likely data errors)
            temp_delta_q99 = temp_delta.quantile(0.99)
            temp_delta_q01 = temp_delta.quantile(0.01)
            tint_delta_q99 = tint_delta.quantile(0.99)
            tint_delta_q01 = tint_delta.quantile(0.01)

            print(f"  Removing outliers outside: Temp [{temp_delta_q01:.0f}, {temp_delta_q99:.0f}], Tint [{tint_delta_q01:.0f}, {tint_delta_q99:.0f}]")

            # Clip extreme values
            temp_delta = temp_delta.clip(temp_delta_q01, temp_delta_q99)
            tint_delta = tint_delta.clip(tint_delta_q01, tint_delta_q99)

            y = pd.DataFrame({
                'temp_delta': temp_delta,
                'tint_delta': tint_delta
            })

            # Store current values and clipping bounds
            self.current_temp = sliders_df['currTemp'].copy()
            self.current_tint = sliders_df['currTint'].copy()
            self.temp_delta_min = temp_delta_q01
            self.temp_delta_max = temp_delta_q99
            self.tint_delta_min = tint_delta_q01
            self.tint_delta_max = tint_delta_q99
        else:
            print("\n⚠️  currTemp/currTint not found, using absolute prediction")
            self.predict_deltas = False
            y = sliders_df[['Temperature', 'Tint']].copy()

        ids = sliders_df['id_global'].values

        # Store statistics
        for col in X_meta.columns:
            if X_meta[col].dtype in ['float64', 'int64']:
                self.feature_means[col] = X_meta[col].mean()
                self.feature_medians[col] = X_meta[col].median()

        # Encode categorical features
        categorical_cols = X_meta.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            self.label_encoders[col] = LabelEncoder()
            X_meta[col] = self.label_encoders[col].fit_transform(X_meta[col].astype(str))
            self.feature_means[col] = X_meta[col].mode()[0] if len(X_meta[col].mode()) > 0 else 0

        # Load images
        print("Loading images...")
        images = []
        valid_indices = []

        sample_id = ids[0]
        possible_extensions = ['.tiff', '.tif', '.TIFF', '.TIF', '.jpg', '.JPG', '.png', '.PNG']
        actual_extension = None

        for ext in possible_extensions:
            if os.path.exists(f'{train_path}/images/{sample_id}{ext}'):
                actual_extension = ext
                print(f"Detected image extension: {ext}")
                break

        if actual_extension is None:
            raise ValueError("Could not detect image extension")

        for idx, img_id in enumerate(ids):
            img_path = f'{train_path}/images/{img_id}{actual_extension}'
            if os.path.exists(img_path):
                try:
                    img = Image.open(img_path)
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    img = img.resize((224, 224))
                    img_array = np.array(img).astype(np.float32) / 255.0
                    images.append(img_array)
                    valid_indices.append(idx)

                    if (idx + 1) % 500 == 0:
                        print(f"Loaded {idx + 1}/{len(ids)} images")
                except Exception as e:
                    print(f"Error loading {img_id}: {e}")

        images = np.array(images)
        X_meta = X_meta.iloc[valid_indices].reset_index(drop=True)
        y = y.iloc[valid_indices].reset_index(drop=True)

        if self.predict_deltas:
            self.current_temp = self.current_temp.iloc[valid_indices].reset_index(drop=True)
            self.current_tint = self.current_tint.iloc[valid_indices].reset_index(drop=True)

        print(f"Successfully loaded {len(images)} images")

        if self.predict_deltas:
            print(f"\nDelta statistics:")
            print(f"  Temp delta range: [{y['temp_delta'].min():.0f}, {y['temp_delta'].max():.0f}]")
            print(f"  Temp delta mean: {y['temp_delta'].mean():.1f}, std: {y['temp_delta'].std():.1f}")
            print(f"  Tint delta range: [{y['tint_delta'].min():.0f}, {y['tint_delta'].max():.0f}]")
            print(f"  Tint delta mean: {y['tint_delta'].mean():.1f}, std: {y['tint_delta'].std():.1f}")

        return images, X_meta, y, ids[valid_indices]

    def build_image_model(self, input_shape=(224, 224, 3)):
        """Lightweight CNN for image features"""
        model = keras.Sequential([
            layers.Input(shape=input_shape),

            layers.Conv2D(32, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(32, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(2),
            layers.Dropout(0.25),

            layers.Conv2D(64, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(64, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(2),
            layers.Dropout(0.25),

            layers.Conv2D(128, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.GlobalAveragePooling2D(),

            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Dense(128, activation='relu'),
        ])

        return model

    def build_fusion_model(self, image_features_dim, metadata_dim):
        """Fusion model optimized for delta prediction"""
        # Image branch
        image_input = layers.Input(shape=(image_features_dim,))
        x_img = layers.Dense(256, activation='relu')(image_input)
        x_img = layers.BatchNormalization()(x_img)
        x_img = layers.Dropout(0.3)(x_img)

        # Metadata branch (currTemp/currTint are important here!)
        metadata_input = layers.Input(shape=(metadata_dim,))
        x_meta = layers.Dense(128, activation='relu')(metadata_input)
        x_meta = layers.BatchNormalization()(x_meta)
        x_meta = layers.Dropout(0.3)(x_meta)
        x_meta = layers.Dense(64, activation='relu')(x_meta)

        # Fusion
        combined = layers.concatenate([x_img, x_meta])

        x = layers.Dense(256, activation='relu')(combined)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.4)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)

        # Separate heads for temp_delta and tint_delta
        temp_head = layers.Dense(64, activation='relu')(x)
        temp_head = layers.Dropout(0.2)(temp_head)
        temp_output = layers.Dense(1, name='temp_delta')(temp_head)

        tint_head = layers.Dense(64, activation='relu')(x)
        tint_head = layers.Dropout(0.2)(tint_head)
        tint_output = layers.Dense(1, name='tint_delta')(tint_head)

        output = layers.concatenate([temp_output, tint_output])

        model = keras.Model(inputs=[image_input, metadata_input], outputs=output)
        return model

    def train(self, train_path='/content/aftershoot_extracted/dataset/Train',
              epochs=80, batch_size=32):
        """Train with delta-based approach"""
        images, X_meta, y, ids = self.load_and_preprocess_data(train_path)

        # Split data
        if self.predict_deltas:
            X_img_train, X_img_val, X_meta_train, X_meta_val, y_train, y_val, curr_temp_train, curr_temp_val, curr_tint_train, curr_tint_val = train_test_split(
                images, X_meta, y, self.current_temp, self.current_tint,
                test_size=0.15, random_state=42
            )
        else:
            X_img_train, X_img_val, X_meta_train, X_meta_val, y_train, y_val = train_test_split(
                images, X_meta, y, test_size=0.15, random_state=42
            )

        # Scale metadata
        X_meta_train_scaled = self.scaler.fit_transform(X_meta_train)
        X_meta_val_scaled = self.scaler.transform(X_meta_val)

        print("Training image feature extractor...")
        self.image_model = self.build_image_model()

        print("Extracting image features...")
        img_features_train = self.image_model.predict(X_img_train, batch_size=batch_size, verbose=0)
        img_features_val = self.image_model.predict(X_img_val, batch_size=batch_size, verbose=0)

        print("Training fusion model...")
        self.ensemble_model = self.build_fusion_model(
            img_features_train.shape[1],
            X_meta_train_scaled.shape[1]
        )

        # Loss function - MAE is good for deltas
        self.ensemble_model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.002),
            loss='mae',
            metrics=['mae']
        )

        early_stop = keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=20, restore_best_weights=True, verbose=1
        )

        reduce_lr = keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7, verbose=1
        )

        history = self.ensemble_model.fit(
            [img_features_train, X_meta_train_scaled],
            y_train.values,
            validation_data=([img_features_val, X_meta_val_scaled], y_val.values),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stop, reduce_lr],
            verbose=1
        )

        # Train gradient boosting
        print("Training gradient boosting ensemble...")
        combined_features_train = np.concatenate([img_features_train, X_meta_train_scaled], axis=1)

        self.metadata_model = MultiOutputRegressor(
            GradientBoostingRegressor(
                n_estimators=300,
                max_depth=6,
                learning_rate=0.05,
                subsample=0.8,
                random_state=42
            )
        )
        self.metadata_model.fit(combined_features_train, y_train)

        # Evaluate both models separately to determine best weights
        val_pred_nn = self.ensemble_model.predict([img_features_val, X_meta_val_scaled], verbose=0)
        combined_features_val = np.concatenate([img_features_val, X_meta_val_scaled], axis=1)
        val_pred_gb = self.metadata_model.predict(combined_features_val)

        if self.predict_deltas:
            # Test different ensemble weights
            best_mae = float('inf')
            best_weight = 0.5

            for nn_weight in [0.3, 0.4, 0.5, 0.6, 0.7]:
                val_pred_delta = nn_weight * val_pred_nn + (1 - nn_weight) * val_pred_gb

                # Clip deltas
                val_pred_delta[:, 0] = np.clip(val_pred_delta[:, 0], self.temp_delta_min, self.temp_delta_max)
                val_pred_delta[:, 1] = np.clip(val_pred_delta[:, 1], self.tint_delta_min, self.tint_delta_max)

                # Convert to absolute
                val_pred_absolute = np.column_stack([
                    curr_temp_val.values + val_pred_delta[:, 0],
                    curr_tint_val.values + val_pred_delta[:, 1]
                ])

                # Clip to valid ranges
                val_pred_absolute[:, 0] = np.clip(val_pred_absolute[:, 0], 2000, 10000)
                val_pred_absolute[:, 1] = np.clip(val_pred_absolute[:, 1], -150, 150)

                # Get ground truth
                y_val_absolute = np.column_stack([
                    curr_temp_val.values + y_val.iloc[:, 0].values,
                    curr_tint_val.values + y_val.iloc[:, 1].values
                ])

                mae_temp = np.mean(np.abs(y_val_absolute[:, 0] - val_pred_absolute[:, 0]))
                mae_tint = np.mean(np.abs(y_val_absolute[:, 1] - val_pred_absolute[:, 1]))
                avg_mae = (mae_temp + mae_tint) / 2

                if avg_mae < best_mae:
                    best_mae = avg_mae
                    best_weight = nn_weight
                    best_temp_mae = mae_temp
                    best_tint_mae = mae_tint

            self.nn_weight = best_weight
            print(f"\n  Best ensemble weight: {best_weight:.1f} NN / {1-best_weight:.1f} GB")

            # Final predictions with best weight
            val_pred_delta = best_weight * val_pred_nn + (1 - best_weight) * val_pred_gb
            val_pred_delta[:, 0] = np.clip(val_pred_delta[:, 0], self.temp_delta_min, self.temp_delta_max)
            val_pred_delta[:, 1] = np.clip(val_pred_delta[:, 1], self.tint_delta_min, self.tint_delta_max)

            mae_temp = best_temp_mae
            mae_tint = best_tint_mae

            # Also show delta MAE
            mae_temp_delta = np.mean(np.abs(y_val.iloc[:, 0].values - val_pred_delta[:, 0]))
            mae_tint_delta = np.mean(np.abs(y_val.iloc[:, 1].values - val_pred_delta[:, 1]))

            print(f"\n{'='*50}")
            print(f"Validation Results (DELTA prediction):")
            print(f"  Delta MAE - Temp: {mae_temp_delta:.2f}, Tint: {mae_tint_delta:.2f}")
            print(f"\nValidation Results (ABSOLUTE values):")
            print(f"  Temperature MAE: {mae_temp:.2f}")
            print(f"  Tint MAE: {mae_tint:.2f}")
            print(f"  Average MAE: {(mae_temp + mae_tint) / 2:.2f}")
            print(f"{'='*50}\n")
        else:
            self.nn_weight = 0.5
            val_pred_delta = 0.5 * val_pred_nn + 0.5 * val_pred_gb
            mae_temp = np.mean(np.abs(y_val.iloc[:, 0].values - val_pred_delta[:, 0]))
            mae_tint = np.mean(np.abs(y_val.iloc[:, 1].values - val_pred_delta[:, 1]))
            print(f"\n{'='*50}")
            print(f"Temperature MAE: {mae_temp:.2f}")
            print(f"Tint MAE: {mae_tint:.2f}")
            print(f"Average MAE: {(mae_temp + mae_tint) / 2:.2f}")
            print(f"{'='*50}\n")

        return history

    def predict(self, val_path='/content/aftershoot_extracted/dataset/Validation',
                output_file='submission.csv'):
        """Generate predictions"""
        sliders_input = pd.read_csv(f'{val_path}/sliders_input.csv')
        ids = sliders_input['id_global'].values

        # Store current values for delta reconstruction
        if self.predict_deltas:
            if 'currTemp' in sliders_input.columns and 'currTint' in sliders_input.columns:
                curr_temp_val = sliders_input['currTemp'].values
                curr_tint_val = sliders_input['currTint'].values
                print(f"Using currTemp/currTint for delta reconstruction")
            else:
                print("⚠️  Warning: currTemp/currTint not in validation, using defaults")
                curr_temp_val = np.full(len(ids), 5500)  # Typical daylight
                curr_tint_val = np.full(len(ids), 0)

        # Handle missing features
        available_cols = [col for col in self.feature_cols if col in sliders_input.columns]
        missing_cols = [col for col in self.feature_cols if col not in sliders_input.columns]

        if missing_cols:
            print(f"Warning: {len(missing_cols)} features missing")

        X_meta = sliders_input[available_cols].copy()

        for col in missing_cols:
            if col in self.feature_medians:
                X_meta[col] = self.feature_medians[col]
            elif col in self.feature_means:
                X_meta[col] = self.feature_means[col]
            else:
                X_meta[col] = 0

        X_meta = X_meta[self.feature_cols]

        # Handle categorical features
        for col in X_meta.select_dtypes(include=['object']).columns:
            if col in self.label_encoders:
                known_categories = set(self.label_encoders[col].classes_)
                X_meta[col] = X_meta[col].astype(str)
                mode_value = self.label_encoders[col].classes_[int(self.feature_means.get(col, 0))]
                X_meta[col] = X_meta[col].apply(lambda x: x if x in known_categories else mode_value)
                X_meta[col] = self.label_encoders[col].transform(X_meta[col])
            else:
                X_meta[col] = 0

        # Load images
        print("Loading validation images...")
        images = []

        sample_id = ids[0]
        possible_extensions = ['.tiff', '.tif', '.TIFF', '.TIF', '.jpg', '.JPG', '.png', '.PNG']
        actual_extension = '.tiff'

        for ext in possible_extensions:
            if os.path.exists(f'{val_path}/images/{sample_id}{ext}'):
                actual_extension = ext
                break

        for idx, img_id in enumerate(ids):
            img_path = f'{val_path}/images/{img_id}{actual_extension}'
            try:
                img = Image.open(img_path)
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                img = img.resize((224, 224))
                img_array = np.array(img).astype(np.float32) / 255.0
                images.append(img_array)

                if (idx + 1) % 100 == 0:
                    print(f"Loaded {idx + 1}/{len(ids)} images")
            except Exception as e:
                print(f"Error loading {img_id}: {e}")
                images.append(np.zeros((224, 224, 3), dtype=np.float32))

        images = np.array(images)
        X_meta_scaled = self.scaler.transform(X_meta)

        # Predict
        print("Generating predictions...")
        img_features = self.image_model.predict(images, batch_size=32, verbose=0)

        pred_nn = self.ensemble_model.predict([img_features, X_meta_scaled], verbose=0)
        combined_features = np.concatenate([img_features, X_meta_scaled], axis=1)
        pred_gb = self.metadata_model.predict(combined_features)

        # Use optimized ensemble weight
        nn_weight = getattr(self, 'nn_weight', 0.5)
        pred_delta = nn_weight * pred_nn + (1 - nn_weight) * pred_gb

        # Clip predicted deltas to reasonable range
        if self.predict_deltas:
            pred_delta[:, 0] = np.clip(pred_delta[:, 0], self.temp_delta_min, self.temp_delta_max)
            pred_delta[:, 1] = np.clip(pred_delta[:, 1], self.tint_delta_min, self.tint_delta_max)

        # Convert deltas to absolute values
        if self.predict_deltas:
            predictions = np.column_stack([
                curr_temp_val + pred_delta[:, 0],
                curr_tint_val + pred_delta[:, 1]
            ])

            # Clip final predictions to valid ranges
            predictions[:, 0] = np.clip(predictions[:, 0], 2000, 10000)  # Temperature: 2000-10000K
            predictions[:, 1] = np.clip(predictions[:, 1], -150, 150)    # Tint: -150 to +150

            print(f"\nDelta statistics (predicted):")
            print(f"  Temp delta mean: {pred_delta[:, 0].mean():.1f}, std: {pred_delta[:, 0].std():.1f}")
            print(f"  Tint delta mean: {pred_delta[:, 1].mean():.1f}, std: {pred_delta[:, 1].std():.1f}")
        else:
            predictions = pred_delta
            # Clip to valid ranges
            predictions[:, 0] = np.clip(predictions[:, 0], 2000, 10000)
            predictions[:, 1] = np.clip(predictions[:, 1], -150, 150)

        predictions = np.round(predictions).astype(int)

        # Create submission
        submission = pd.DataFrame({
            'id_global': ids,
            'Temperature': predictions[:, 0],
            'Tint': predictions[:, 1]
        })

        submission.to_csv(output_file, index=False)
        print(f"\nPredictions saved to {output_file}")

        return submission

# Main execution
if __name__ == "__main__":
    predictor = WhiteBalancePredictor()

    print("=" * 60)
    print("WHITE BALANCE PREDICTOR - Delta-based approach")
    print("=" * 60)

    print("\nStarting training...")
    history = predictor.train(
        train_path='/content/aftershoot_extracted/dataset/Train',
        epochs=80,
        batch_size=16
    )

    print("\nGenerating predictions...")
    submission = predictor.predict(
        val_path='/content/aftershoot_extracted/dataset/Validation',
        output_file='submission.csv'
    )

    print("\nSubmission preview:")
    print(submission.head(10))
    print(f"\nSubmission shape: {submission.shape}")
    print(f"\nTemperature range: [{submission['Temperature'].min()}, {submission['Temperature'].max()}]")
    print(f"Tint range: [{submission['Tint'].min()}, {submission['Tint'].max()}]")
    print("\n✅ Done! Upload submission.csv to the competition platform.")

WHITE BALANCE PREDICTOR - Delta-based approach

Starting training...
Using 7 features: ['aperture', 'flashFired', 'focalLength', 'isoSpeedRating', 'shutterSpeed', 'currTemp', 'currTint']

🎯 Using DELTA prediction (change from current values)
  Removing outliers outside: Temp [-2908, 2470], Tint [-21, 19]
Loading images...
Detected image extension: .tif
Loaded 500/2538 images
Loaded 1000/2538 images
Loaded 1500/2538 images
Loaded 2000/2538 images
Loaded 2500/2538 images
Successfully loaded 2538 images

Delta statistics:
  Temp delta range: [-2908, 2470]
  Temp delta mean: -313.8, std: 921.8
  Tint delta range: [-21, 19]
  Tint delta mean: -0.8, std: 7.8
Training image feature extractor...
Extracting image features...
Training fusion model...
Epoch 1/80
135/135 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 349.9665 - mae: 349.9665 - val_loss: 276.5581 - val_mae: 276.5581 - learning_rate: 0.0020
Epoch 2/80
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 224.4081 - mae: 224.4081 - val_loss: 2